In [2]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np
import sklearn
sklearn.__version__


'1.3.0'

In [3]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('샘플의 수 :',len(documents))

샘플의 수 : 11314


In [4]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

뉴스그룹 데이터에는 특수문자가 포함된 다수의 영어문장으로 구성되어져 있습니다. 

이런 형식의 샘플이 총 11,314개 존재합니다. 

사이킷런이 제공하는 뉴스그룹 데이터에서 target_name에는 본래 이 뉴스그룹 데이터가 

어떤 20개의 카테고리를 갖고있었는지가 저장되어져 있습니다. 이를 출력해보겠습니다.

In [5]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [6]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join( [w for w in x.split() if len(w)>3 ]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [7]:
news_df['clean_doc'][1]

"yeah, expect people read faq, etc. actually accept hard atheism? need little leap faith, jimmy. your logic runs steam! jim, sorry can't pity you, jim. sorry that have these feelings denial about faith need well, just pretend that will happily ever after anyway. maybe start newsgroup, alt.atheist.hard, won't bummin' much? bye-bye, jim. don't forget your flintstone's chewables! bake timmons,"

우선 특수문자가 제거되었으며, if나 you와 같은 길이가 3이하인 단어가 제거된 것을 확인할 수 있습니다. 

뿐만 아니라 대문자가 전부 소문자로 바뀌었습니다. 

이제 뉴스그룹 데이터에서 불용어를 제거합니다. 

불용어를 제거하기 위해서 토큰화를 우선 수행합니다. 

토큰화와 불용어 제거를 순차적으로 진행합니다.

In [8]:
# NLTK로부터 불용어를 받아온다.
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.

In [9]:
print(tokenized_doc[1])

['yeah,', 'expect', 'people', 'read', 'faq,', 'etc.', 'actually', 'accept', 'hard', 'atheism?', 'need', 'little', 'leap', 'faith,', 'jimmy.', 'logic', 'runs', 'steam!', 'jim,', 'sorry', "can't", 'pity', 'you,', 'jim.', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well,', 'pretend', 'happily', 'ever', 'anyway.', 'maybe', 'start', 'newsgroup,', 'alt.atheist.hard,', "bummin'", 'much?', 'bye-bye,', 'jim.', 'forget', "flintstone's", 'chewables!', 'bake', 'timmons,']


In [22]:
print(tokenized_doc[:])

0        [well, sure, story, seem, biased., disagree, s...
1        [yeah,, expect, people, read, faq,, etc., actu...
2        [although, realize, principle, strongest, poin...
3        [notwithstanding, legitimate, fuss, proposal,,...
4        [well,, change, scoring, playoff, pool., unfor...
                               ...                        
11309    [danny, rubenstein,, israeli, journalist,, spe...
11310                                                   []
11311    [agree., home, runs, clemens, always, memorabl...
11312    [used, deskjet, orange, micros, grappler, syst...
11313    [^^^^^^, argument, murphy., scared, hell, came...
Name: clean_doc, Length: 11314, dtype: object


기존에 있었던 불용어에 속하던 your, about, just, that, will, after 단어들이 사라졌을 뿐만 아니라, 토큰화가 수행된 것을 확인할 수 있습니다.

### **3) TF-IDF 행렬 만들기**

불용어 제거를 위해 토큰화 작업을 수행하였지만, TfidfVectorizer(TF-IDF 실습 참고)는 기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용합니다. 그렇기 때문에 TfidfVectorizer를 사용해서 TF-IDF 행렬을 만들기 위해서 다시 토큰화 작업을 역으로 취소하는 작업을 수행해보도록 하겠습니다. 이를 역토큰화(Detokenization)라고 합니다.

In [10]:
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

In [11]:
news_df['clean_doc'][1]

"yeah, expect people read faq, etc. actually accept hard atheism? need little leap faith, jimmy. logic runs steam! jim, sorry can't pity you, jim. sorry feelings denial faith need well, pretend happily ever anyway. maybe start newsgroup, alt.atheist.hard, bummin' much? bye-bye, jim. forget flintstone's chewables! bake timmons,"

정상적으로 불용어가 제거된 상태에서 역토큰화가 수행되었음을 확인할 수 있습니다.

이제 사이킷런의 TfidfVectorizer를 통해 단어 1,000개에 대한 TF-IDF 행렬을 만들 것입니다.

 물론 텍스트 데이터에 있는 모든 단어를 가지고 행렬을 만들 수는 있겠지만, 여기서는 1,000개의 단어로 제한하도록 하겠습니다.

In [12]:
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000, # 상위 1,000개의 단어를 보존
max_df = 0.5, smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

TF-IDF 행렬의 크기 : (11314, 1000)


In [13]:
print(X)

  (0, 687)	0.1296353669652296
  (0, 474)	0.1654349645287957
  (0, 528)	0.12056611615981094
  (0, 129)	0.1353917589343785
  (0, 546)	0.13730094657898326
  (0, 406)	0.13187765371261517
  (0, 734)	0.16560097932778423
  (0, 824)	0.1916531368961578
  (0, 750)	0.17005059429692632
  (0, 211)	0.16429581785105885
  (0, 748)	0.1678393395128178
  (0, 731)	0.1334001116346078
  (0, 894)	0.09142875717846821
  (0, 506)	0.1747572305861188
  (0, 986)	0.12651035082406945
  (0, 469)	0.35795726654853505
  (0, 563)	0.6872510148195169
  (0, 848)	0.1587201954825918
  (0, 854)	0.163349948814478
  (0, 867)	0.11600928494264434
  (1, 80)	0.22175666279878037
  (1, 610)	0.2077273970865982
  (1, 844)	0.16953640462205893
  (1, 559)	0.16291087962121
  (1, 828)	0.37072745091156617
  :	:
  (11313, 219)	0.12164437858388767
  (11313, 240)	0.1579505998623282
  (11313, 909)	0.16851830379964058
  (11313, 155)	0.14375875155695303
  (11313, 943)	0.14124328059721142
  (11313, 547)	0.14358487860740204
  (11313, 714)	0.138312604

### **4) 토픽 모델링(Topic Modeling)**

이제 TF-IDF 행렬을 다수의 행렬로 분해해보도록 하겠습니다. 여기서는 사이킷런의 절단된 SVD(Truncated SVD)를 사용합니다. 절단된 SVD를 사용하면 차원을 축소할 수 있습니다. 원래 기존 뉴스그룹 데이터가 20개의 카테고리를 갖고있었기 때문에, 20개의 토픽을 가졌다고 가정하고 토픽 모델링을 시도해보겠습니다. 토픽의 숫자는 n_components의 파라미터로 지정이 가능합니다.

In [14]:
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

20

In [15]:
print(type(svd_model))

<class 'sklearn.decomposition._truncated_svd.TruncatedSVD'>


여기서 svd_model.componets_는 앞서 배운 LSA에서 VT에 해당됩니다.

In [16]:
np.shape(svd_model.components_)

(20, 1000)

정확하게 토픽의 수 t × 단어의 수의 크기를 가지는 것을 볼 수 있습니다.

In [18]:
terms = vectorizer.get_feature_names_out() # 단어 집합. 1,000개의 단어가 저장됨. 
# When sklearn.__version__ >= 1.0.x use following method

print(terms)

['00' '000' '01' '02' '04' '0d' '0t' '10' '100' '11' '12' '13' '14' '145'
 '15' '16' '17' '18' '19' '1990' '1991' '1992' '1993' '1d9' '1t' '20' '21'
 '22' '23' '24' '25' '27' '2di' '2tm' '30' '32' '34' '34u' '3t' '40' '45'
 '50' '500' '55' '6ei' '6um' '75' '75u' '7ey' '7u' '80' '800' '90' '91'
 '92' '93' '9v' 'a86' 'ability' 'able' 'ac' 'accept' 'access' 'according'
 'account' 'action' 'actually' 'added' 'addition' 'address'
 'administration' 'advance' 'ago' 'agree' 'ah' 'air' 'algorithm' 'allow'
 'allowed' 'allows' 'alt' 'amendment' 'america' 'american' 'americans'
 'analysis' 'anonymous' 'answer' 'answers' 'anti' 'anybody' 'apparently'
 'appear' 'appears' 'apple' 'application' 'applications' 'apply'
 'appreciate' 'appreciated' 'approach' 'appropriate' 'april' 'arab'
 'archive' 'area' 'areas' 'argument' 'arguments' 'armenia' 'armenian'
 'armenians' 'arms' 'army' 'article' 'articles' 'asked' 'asking' 'assume'
 'atheism' 'atheists' 'attack' 'attempt' 'au' 'author' 'authority'
 'availabl

In [19]:
def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5))for i in topic.argsort()[:-n - 1:-1]])

In [20]:
get_topics(svd_model.components_, terms)

Topic 1: [('like', 0.2085), ('know', 0.19656), ('people', 0.1912), ('think', 0.17523), ('good', 0.14902)]
Topic 2: [('thanks', 0.31338), ('windows', 0.27934), ('card', 0.17289), ('drive', 0.16141), ('mail', 0.14507)]
Topic 3: [('game', 0.36553), ('team', 0.3133), ('year', 0.28465), ('games', 0.23048), ('season', 0.17026)]
Topic 4: [('edu', 0.50341), ('thanks', 0.25409), ('mail', 0.1758), ('com', 0.11498), ('email', 0.11166)]
Topic 5: [('edu', 0.49934), ('drive', 0.24972), ('com', 0.10645), ('sale', 0.10616), ('soon', 0.09199)]
Topic 6: [('drive', 0.40102), ('thanks', 0.34667), ('know', 0.27592), ('scsi', 0.13765), ('mail', 0.11332)]
Topic 7: [('chip', 0.21565), ('government', 0.20249), ('like', 0.17148), ('encryption', 0.14654), ('clipper', 0.14478)]
Topic 8: [('like', 0.64668), ('edu', 0.31439), ('bike', 0.12683), ('know', 0.12403), ('think', 0.11547)]
Topic 9: [('card', 0.3572), ('sale', 0.17543), ('00', 0.17496), ('video', 0.16994), ('good', 0.15574)]
Topic 10: [('card', 0.45093), (

In [21]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased., disagree, s...
1    [yeah,, expect, people, read, faq,, etc., actu...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal,,...
4    [well,, change, scoring, playoff, pool., unfor...
Name: clean_doc, dtype: object

이제 각 단어에 정수 인코딩을 하는 동시에, **각 뉴스에서의 단어의 빈도수**를 기록해보겠습니다. 

여기서는 각 단어를 **(word_id, word_frequency)의 형태**로 바꾸고자 합니다. 

word_id는 단어가 **정수 인코딩된 값**이고, word_frequency는 **해당 뉴스에서의 해당 단어의 빈도수**를 의미합니다. 

이는 gensim의 corpora.Dictionary()를 사용하여 손쉽게 구할 수 있습니다. 

전체 뉴스에 대해서 정수 인코딩을 수행하고, 두번째 뉴스를 출력해봅시다.

In [29]:
from gensim import corpora

dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text)for text in tokenized_doc]

print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 2), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 2), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1)]


두번째 뉴스의 출력 결과를 봅시다. 

위의 출력 결과 중에서 (66, 2)는 정수 인코딩이 66으로 할당된 단어가 두번째 뉴스에서는 두 번 등장하였음을 의미합니다. 

66이라는 값을 가지는 단어가 정수 인코딩이 되기 전에는 어떤 단어였는지 확인하여봅시다. 

---

이는 dictionary[]에 기존 단어가 무엇인지 알고자하는 정수값을 입력하여 확인할 수 있습니다.

In [30]:
print(dictionary[66])

bye-bye,


In [31]:
len(dictionary)

181856

### **2) LDA 모델 훈련시키기**

기존의 뉴스 데이터가 총 20개의 카테고리를 가지고 있었으므로 토픽의 개수를 20으로 하여 LDA 모델을 학습시켜보도록 하겠습니다.

In [34]:
import gensim
NUM_TOPICS = 20 # 20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.007*"stats" + 0.005*"kent" + 0.004*"symbol" + 0.003*"aged"')
(1, '0.016*"55.0" + 0.003*"terminals" + 0.002*"accelerators" + 0.002*"etc?"')
(2, '0.003*"resources." + 0.003*"cross" + 0.002*"adobe" + 0.002*"countersteering"')
(3, '0.016*"available" + 0.009*"image" + 0.008*"version" + 0.007*"data"')
(4, '0.217*"max>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'" + 0.006*"wiring" + 0.005*"ansi" + 0.003*"francis"')
(5, '0.012*"people" + 0.012*"would" + 0.006*"think" + 0.006*"many"')
(6, '0.011*"period" + 0.010*"power" + 0.006*"---------------" + 0.006*"scorer"')
(7, '0.016*"space" + 0.008*"university" + 0.008*"national" + 0.007*"1993"')
(8, '0.010*"israel" + 0.009*"israeli" + 0.008*"jews" + 0.007*"said,"')
(9, '0.012*"jesus" + 0.012*"greek" + 0.004*"holy" + 0.004*"spirit"')
(10, '0.005*"encryption" + 0.005*"technology" + 0.004*"administration" + 0.004*"copies)"')
(11, '0.008*"would" + 0.007*"like" + 0.006*"know" + 0.006*"using"')
(12, '0.028*"drive" + 0.015*"

각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여줍니다. 

또한 맨 앞에 있는 **토픽 번호**는 0부터 시작하므로 총 20개의 토픽은 0부터 19까지의 번호가 할당되어져 있습니다. 

**passes는 알고리즘의 동작 횟수**를 말하는데, 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 충분히 적당한 횟수를 정해주면 됩니다. 여기서는 총 15회를 수행하였습니다. 

여기서는 num_words=4로 총 **4개의 단어만 출력**하도록 하였습니다. 

만약 10개의 단어를 출력하고 싶다면 아래의 코드를 수행하면 됩니다.

In [35]:
print(ldamodel.print_topics())


[(0, '0.007*"stats" + 0.005*"kent" + 0.004*"symbol" + 0.003*"aged" + 0.003*"fatal" + 0.003*"wires" + 0.003*"cheers," + 0.003*"baseball." + 0.002*"fleet" + 0.002*"wrist"'), (1, '0.016*"55.0" + 0.003*"terminals" + 0.002*"accelerators" + 0.002*"etc?" + 0.001*"disagreement" + 0.001*"(frank" + 0.001*"similarity" + 0.001*"object," + 0.001*"neural" + 0.001*"generated,"'), (2, '0.003*"resources." + 0.003*"cross" + 0.002*"adobe" + 0.002*"countersteering" + 0.002*".bmp" + 0.002*"cardinals" + 0.002*"from?" + 0.002*"clip" + 0.002*"ninth" + 0.002*"overtime"'), (3, '0.016*"available" + 0.009*"image" + 0.008*"version" + 0.007*"data" + 0.007*"also" + 0.007*"server" + 0.007*"software" + 0.007*"window" + 0.007*"anonymous" + 0.005*"subject:"'), (4, '0.217*"max>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'ax>\'" + 0.006*"wiring" + 0.005*"ansi" + 0.003*"francis" + 0.003*"neutral" + 0.003*"part" + 0.002*"--------" + 0.002*"grounded" + 0.002*"------------" + 0.002*"outlet"'), (5, '0.012

### **3) LDA 시각화 하기**

LDA 시각화를 위해서는 pyLDAvis의 설치가 필요합니다. 윈도우의 명령 프롬프트나 MAC/UNIX의 터미널에서 아래의 명령을 수행하여 pyLDAvis를 설치하시기 바랍니다.

In [ ]:
pip install pyLDAvis

In [37]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

좌측의 원들은 각각의 20개의 토픽을 나타냅니다. 각 원과의 거리는 각 토픽들이 서로 얼마나 다른지를 보여줍니다. 

만약 두 개의 원이 겹친다면, 이 두 개의 토픽은 유사한 토픽이라는 의미입니다. 

위의 그림에서는 10번 토픽을 클릭하였고, 이에 따라 우측에는 10번 토픽에 대한 정보가 나타납니다. 

한 가지 주의할 점은 LDA 모델의 출력 결과에서는 토픽 번호가 0부터 할당되어 0~19의 숫자가 사용된 것과는 달리 

위의 LDA 시각화에서는 토픽의 번호가 1부터 시작하므로 각 토픽 번호는 이제 +1이 된 값인 1~20까지의 값을 가집니다.

### **4) 문서 별 토픽 분포 보기**

위에서 **토픽 별 단어 분포는 확인하였으나**, 아직 **문서 별 토픽 분포**에 대해서는 확인하지 못 하였습니다. 

우선 문서 별 토픽 분포를 확인하는 방법을 보겠습니다. 

각 문서의 토픽 분포는 이미 훈련된 LDA 모델인 ldamodel[]에 전체 데이터가 **정수 인코딩 된 결과를 넣은 후**에 확인이 가능합니다. 

여기서는 책의 지면의 한계로 상위 5개의 문서에 대해서만 토픽 분포를 확인해보겠습니다.

In [41]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)


0 번째 문서의 topic 비율은 [(5, 0.41004026), (8, 0.13869469), (11, 0.36545846), (19, 0.07361239)]
1 번째 문서의 topic 비율은 [(3, 0.08535655), (5, 0.7263069), (6, 0.1680192)]
2 번째 문서의 topic 비율은 [(3, 0.08118402), (5, 0.486559), (8, 0.18865976), (11, 0.11513455), (17, 0.09831383), (19, 0.018244741)]
3 번째 문서의 topic 비율은 [(5, 0.454223), (11, 0.3091022), (17, 0.09980917), (19, 0.12521477)]
4 번째 문서의 topic 비율은 [(11, 0.17250255), (15, 0.27046126), (17, 0.39341658), (19, 0.13393883)]


In [70]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table._append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [71]:
topictable = make_topictable_per_doc(ldamodel, corpus)


In [72]:
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,5,0.4101,"[(5, 0.41006064), (8, 0.13869494), (11, 0.3654..."
1,1,5,0.7263,"[(3, 0.08535281), (5, 0.7263105), (6, 0.168019..."
2,2,5,0.4866,"[(3, 0.08118399), (5, 0.48655292), (8, 0.18865..."
3,3,5,0.4542,"[(5, 0.45422345), (11, 0.30908754), (17, 0.099..."
4,4,17,0.3935,"[(11, 0.17245057), (15, 0.27045047), (17, 0.39..."
5,5,5,0.6397,"[(5, 0.6396592), (15, 0.3193193)]"
6,6,11,0.4064,"[(3, 0.023654675), (4, 0.025840325), (5, 0.153..."
7,7,5,0.4272,"[(5, 0.42720982), (8, 0.34881312), (11, 0.0655..."
8,8,17,0.3247,"[(4, 0.2316272), (5, 0.10561723), (9, 0.051837..."
9,9,17,0.3712,"[(5, 0.16999468), (6, 0.0144261485), (9, 0.013..."
